_This notebook started out as a direct iPython translation of the **M7 PDR05 Excel Spreadsheet**, implemented in a similar way as originally done by **Juande Santander-Vela** in his separate analysis of the M7 Milestone PDR05 (MS Word) document._

_This is an interactive document. **sympy** and **astropy** are required to evaluate some of the blocks of code. While overlapping in scope with "Full Computational Requirements for SDP.ipynb" and "Parametric Models of SDP Compute Requirements.ipynb", this document was independently derived._ 

_Equations may be evaluated for the different telescopes and imaging modes, either numerically or symbolically_ 

_The owner of this notebook document is **Francois Malan** of Space Advisory Company._

# 2. Load Implementation & Environment (refer to .py source files):

In [1]:
'''
The following command imports the python modules, methods and code fragments that are required to run this notebook. 
Please refer to the following python files for implementation details (omitted here for readability)
* env_setup.py -- code for setting up the python environment
* parameter_definitions.py -- contains definitions of variables, primary telescope parameters  
* formulae.py -- contains formulae for defiving secondary telescope-specific parameters from input parameters
* implementation.py -- contains methods for performing computations (i.e. crunching the numbers)
'''
from env_setup import *
from parameter_definitions import *
from formulae import *
from implementation import Implementation as imp
from api import SKAAPI as api


IPython console for SymPy 0.7.5 (Python 2.7.9-64-bit) (ground types: python)


In [3]:
import time
t0 = time.time()
telescope = 'SKA1Mid_old'
band = 'Mid1'
mode = 'Spectral'
telescope_parameters = imp.calc_tel_params(telescope=telescope, band=band, mode=mode)
(Tsnap_opt, Nfacet_opt) = imp.find_optimal_Tsnap_Nfacet(telescope_parameters, verbose=True)
print (Tsnap_opt, Nfacet_opt)
#(Tsnap_opt, Nfacet_opt) = (87.16, 3)
t1 = time.time()
expression = telescope_parameters.Rflop
expression_subst = expression.subs({telescope_parameters.Tsnap : Tsnap_opt, telescope_parameters.Nfacet : Nfacet_opt})
result = imp.evaluate_binned_expression(expression_subst, telescope_parameters)
print 'Result of expression = %f Peta' % (result/1e15)
t2 = time.time()
print 'Optimization took %f seconds' % (t1-t0)
print 'Evaluation took %f seconds' % (t2-t1)
print 'Total time = %f seconds' % (t2-t0)

USING BASELINE DEPENDENT TIME AVERAGING
Tdump_predict = Min(1.2*s, 20562.399438421*m*s/Bmax,bin)
Tdump_backward = Min(60.0*s, 20562.399438421*N_facet*m*s/Bmax,bin)
Evaluating Nfacets = 1
Tsnap has been optimized as : 583.293071. (Cost function = 68.352026)
Evaluating Nfacets = 2
Tsnap has been optimized as : 1128.097251. (Cost function = 34.862856)
Evaluating Nfacets = 3
Tsnap has been optimized as : 1488.150440. (Cost function = 27.469010)
Evaluating Nfacets = 4
Tsnap has been optimized as : 1756.085062. (Cost function = 25.237816)
Evaluating Nfacets = 5
Tsnap has been optimized as : 1973.961783. (Cost function = 25.170199)
Evaluating Nfacets = 6
Tsnap has been optimized as : 2160.198443. (Cost function = 26.418773)

Expression increasing with number of facets; aborting exploration of Nfacets > 6

25.170199 PetaFLOPS was the lowest FLOP value, found for (Nfacet, Tsnap) = (5, 1973.96)
(1973.96178262361, 5)
Result of expression = 25.170199 Peta
Optimization took 4.634304 seconds
Evaluat

# 3. Computations:

## Optimizing Tsnap and Nfacet for a given band and mode:

## Looping through a list of (standard) bands -- this may take several minutes:

In [4]:
#selected_bands = ('Low', 'Mid1', 'Sur1')  # Only loop over the first bands for each telescope (assumed to be the most demanding)
#selected_bands = ('SKA1_Low', 'SKA1_Mid', 'SKA1_Low_OLD', 'SKA1_Mid_OLD')  # Only loop over the first bands for each telescope (assumed to be the most demanding)
#selected_modes = ('Continuum', 'Spectral', 'SlowTrans')
#selected_bands = (Bands.Low, Bands.Mid1, Bands.Sur1)  
selected_bands = (Bands.Low, Bands.Low1old, Bands.Mid1, Bands.Mid1old)  
selected_modes = (ImagingModes.Continuum, ImagingModes.Spectral, ImagingModes.SlowTrans)
print "telescope =", telescope
non_peta_values = ('Npix_linear')
values_to_take_max = ('Npix_linear')

results = {}

for band in selected_bands:
    for mode in selected_modes:
        print "telescope before tp call", telescope
        tp = imp.calc_tel_params(telescope=telescope, band=band, mode=mode) #this is broken - mustn't include telescope in this call. 
        #Telescope name must be inferred from the band only but here it's out of sequence - potentially we need to re-order the call in calc tel params to applt band params before applying telescope params
        
        telescope=tp.telescope
        print "telescope after tp call", telescope
        telescope=tp.telescope
        tp = imp.calc_tel_params(telescope=telescope, band=band, mode=mode)
         
        print '.'
        print '>> Evaluating %s band %s in %s mode' % (tp.telescope, band, mode)
        (Tsnap_opt, Nfacet_opt) = imp.find_optimal_Tsnap_Nfacet(tp)
        print 'Optimal (Tsnap, Nfacet) values = (%.2f sec, %d)' % (Tsnap_opt, Nfacet_opt)
        
        parameters = imp.calc_tel_params(telescope=tp.telescope, band=band, mode=mode)
        substitution = {parameters.Tsnap : Tsnap_opt, parameters.Nfacet : Nfacet_opt}
        
        expressions = (tp.Rflop_conv/1e15, tp.Rflop_fft/1e15, tp.Rflop_grid/1e15, tp.Rflop_proj/1e15, tp.Rflop_phrot/1e15, tp.Rflop/1e15, tp.Mbuf_vis/1e15, tp.Mw_cache/1e15, tp.Npix_linear, tp.Rio/1e15)
        expression_strings = ('Rflop_conv', 'Rflop_fft', 'Rflop_grid', 'Rflop_proj', 'Rflop_phrot', 'Rflop','Mbuf_vis', 'Mw_cache', 'Npix_linear', 'Rio')
        key_expressions = zip(expression_strings, expressions)
        
        for key_expression in key_expressions: 
            key = key_expression[0]
            expression = key_expression[1]
            expression_subst = expression.subs(substitution)
            take_max = (key in values_to_take_max)                
            result = imp.evaluate_binned_expression(expression_subst, parameters, take_max=take_max)
            results[(band, mode, key)] = result
            
            if key in non_peta_values:
                print '-> %s : %s = %g' % (mode, key, result)
            else:    
                print '-> %s : %s = %.3g Peta' % (mode, key, result)
                
print '\n>>> Done!'

telescope = SKA1Mid_old
telescope before tp call SKA1Mid_old
USING BASELINE DEPENDENT TIME AVERAGOG
telescope after tp call SKA1Low_rebaselined
USING BASELINE DEPENDENT TIME AVERAGOG
.
>> Evaluating SKA1Low_rebaselined band Low in Continuum mode
Optimal (Tsnap, Nfacet) values = (58.75 sec, 2)
USING BASELINE DEPENDENT TIME AVERAGOG
-> Continuum : Rflop_conv = 0.38 Peta
-> Continuum : Rflop_fft = 0.214 Peta
-> Continuum : Rflop_grid = 0.14 Peta
-> Continuum : Rflop_proj = 0.152 Peta
-> Continuum : Rflop_phrot = 0.00355 Peta
-> Continuum : Rflop = 0.889 Peta
-> Continuum : Mbuf_vis = 0.919 Peta
-> Continuum : Mw_cache = 0.00023 Peta
-> Continuum : Npix_linear = 16719.5
-> Continuum : Rio = 0.000936 Peta
telescope before tp call SKA1Low_rebaselined
USING BASELINE DEPENDENT TIME AVERAGOG
telescope after tp call SKA1Low_rebaselined
USING BASELINE DEPENDENT TIME AVERAGOG
.
>> Evaluating SKA1Low_rebaselined band Low in Spectral mode
Optimal (Tsnap, Nfacet) values = (122.67 sec, 1)
USING BASELI

## Because it takes a little while to compute all results, let's just dump them out to screen:

In [8]:
results

{('Low', 'Continuum', 'Mbuf_vis'): 0.919299454404957,
 ('Low', 'Continuum', 'Mw_cache'): 0.00022995559910016157,
 ('Low', 'Continuum', 'Npix_linear'): 16719.454136008026,
 ('Low', 'Continuum', 'Rflop'): 0.8892945825221297,
 ('Low', 'Continuum', 'Rflop_conv'): 0.3799487005701146,
 ('Low', 'Continuum', 'Rflop_fft'): 0.21361539471259547,
 ('Low', 'Continuum', 'Rflop_grid'): 0.13991939891411012,
 ('Low', 'Continuum', 'Rflop_phrot'): 0.0035466799938462844,
 ('Low', 'Continuum', 'Rflop_proj'): 0.1522644083314633,
 ('Low', 'Continuum', 'Rio'): 0.0009363235183754192,
 ('Low', 'SlowTrans', 'Mbuf_vis'): 1.7780350519432177e-05,
 ('Low', 'SlowTrans', 'Mw_cache'): 2.097209798205967e-06,
 ('Low', 'SlowTrans', 'Npix_linear'): 5015.836240802408,
 ('Low', 'SlowTrans', 'Rflop'): 0.09712664325478536,
 ('Low', 'SlowTrans', 'Rflop_conv'): 0.012134417832314292,
 ('Low', 'SlowTrans', 'Rflop_fft'): 0.08246842139404671,
 ('Low', 'SlowTrans', 'Rflop_grid'): 0.002400329372039419,
 ('Low', 'SlowTrans', 'Rflop_phr

## Now we can summarize (by summing) them for the three telescopes:

In [9]:
summable_expressions = {'Rflop_conv', 'Rflop_fft', 'Rflop_grid', 'Rflop_proj', 'Mbuf_vis', 'Rflop', 'Mw_cache', 'Rio'}
imaging_modes = {'Spectral', 'Continuum', 'SlowTrans'}

results_per_band = {}
for band in selected_bands:
    #tel = band_info[band]['telescope']
    if not band in results_per_band:
        results_per_band[band] = {}
    for expression in summable_expressions:
        if not expression in results_per_band[band]:
            results_per_band[band][expression] = 0
        for mode in imaging_modes:
            assert (band, mode, expression) in results
            results_per_band[band][expression] += results[(band, mode, expression)]

results_per_band

{'Low': {'Mbuf_vis': 16.19605510696676,
  'Mw_cache': 0.01229714000311212,
  'Rflop': 3.212016713741261,
  'Rflop_conv': 1.0278874743771902,
  'Rflop_fft': 0.9218954206122479,
  'Rflop_grid': 0.6650039512460884,
  'Rflop_proj': 0.5935597128555038,
  'Rio': 0.0018796618321561237},
 'Low1old': {'Mbuf_vis': 242.212318234133,
  'Mw_cache': 0.046262466792076726,
  'Rflop': 26.625254460869,
  'Rflop_conv': 9.056635775184626,
  'Rflop_fft': 6.585337476976042,
  'Rflop_grid': 6.536127409983484,
  'Rflop_proj': 4.433527035424371,
  'Rio': 0.01746756058651658},
 'Mid1': {'Mbuf_vis': 4.327100305465612,
  'Mw_cache': 0.04384037074124114,
  'Rflop': 7.848278821610856,
  'Rflop_conv': 2.8141905379227454,
  'Rflop_fft': 3.5161088611946587,
  'Rflop_grid': 0.31280194475970646,
  'Rflop_proj': 1.15315701112228,
  'Rio': 0.015720120543819653},
 'Mid1old': {'Mbuf_vis': 24.157106779875505,
  'Mw_cache': 0.12209746283842159,
  'Rflop': 32.55802649254882,
  'Rflop_conv': 15.109166635304565,
  'Rflop_fft': 1

## Let's plot the FLOP values

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 8, 6  # that's default image size for this interactive session

results_per_tel = results_per_band
t = results_per_tel.keys()
t.sort()
for tel in t:
    values = results_per_tel[tel]

    # The slices will be ordered and plotted counter-clockwise.
    labels = '(de)Gridding', '(i)FFT', '(Re)Projection', 'Conv Kernel Calculation'
    sizes = [values['Rflop_grid'], values['Rflop_fft'], values['Rflop_proj'], values['Rflop_conv']]
    colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
    explode = (0.05, 0.05, 0.05, 0.05) # Radial offset of the slices

    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    # Set aspect ratio to be equal so that pie is drawn as a circle.
    plt.axis('equal')
    plt.title('%s\n' % tel)

    plt.show()
    
results_per_param = {}
for tel in t:
    for param in results_per_tel[tel]:
        if not param in results_per_param:
            results_per_param[param] = []
        results_per_param[param].append(results_per_tel[tel][param])

#print results_per_param
ind = np.arange(4)

p = 'Rflop_grid'
res_grid = (results_per_tel[t[0]][p], results_per_tel[t[1]][p], results_per_tel[t[2]][p], results_per_tel[t[3]][p])
p = 'Rflop_fft'
res_fft = (results_per_tel[t[0]][p], results_per_tel[t[1]][p], results_per_tel[t[2]][p], results_per_tel[t[3]][p])
p = 'Rflop_proj'
res_proj = (results_per_tel[t[0]][p], results_per_tel[t[1]][p], results_per_tel[t[2]][p], results_per_tel[t[3]][p])
p = 'Rflop_conv'
res_conv = (results_per_tel[t[0]][p], results_per_tel[t[1]][p], results_per_tel[t[2]][p], results_per_tel[t[3]][p])

#Plot a stacked bar chart
width = 0.6
val = np.array(res_grid)
p1 = plt.bar(ind, val, width, color=colors[0])
bottoms = val
val = np.array(res_fft)
p2 = plt.bar(ind, val, width, color=colors[1],
             bottom=bottoms)
bottoms += val
val = np.array(res_proj)
p3 = plt.bar(ind, val, width, color=colors[2],
             bottom=bottoms)
bottoms += val
val = np.array(res_conv)
p4 = plt.bar(ind, val, width, color=colors[3],
             bottom=bottoms)


plt.xticks(ind+width/2., t )
plt.title('PetaFLOPS')
plt.legend(labels, loc=2) # legend upper-left

NameError: name 'results_per_band' is not defined